In [1]:
import requests
import fitz
import io
import re
from bs4 import BeautifulSoup as bs
import requests
import metadata_parser as mp
import pandas as pd

In [2]:
acl = pd.read_excel("./paper-links/PaperLinks_ACL.xlsx")

In [ ]:
# Get the paper metadata
list_year, list_title, list_conf_jour, list_authors, list_keywords, list_abstract = [], [], [], [], [], []
index = 1
for paper_url in acl["Paper Links"]:
  paper_url = re.sub(".pdf","",paper_url)
  try:
    # Get page
    page = mp.MetadataParser(paper_url)
    page_bs = requests.get(paper_url)
    parsed_bs = bs(page_bs.text,'html.parser')   
    try:
      # Get Year
      year = (page.get_metadatas('citation_publication_date'))
      year = year[0].split("/")[0]
    except:
      year = "failed_to_scrap"   
    try:
      # Get Title
      title = page.get_metadatas('citation_title')
      title = title[0]
    except:
      title = "failed_to_scrap"  
    try:
      # Get Conf/Journal Name
      conf_jour_name = (page.get_metadatas('citation_conference_title'))
      conf_jour_name = conf_jour_name[0]
    except:
      conf_jour_name = "failed_to_scrap"   
    try:
      # Get Authors
      authors = (page.get_metadatas('citation_author'))
      authors = ", ".join(authors)
    except:
      authors = "failed_to_scrap"
    # Assign Keywords (ACL does not provide keywords)
    keywords = "no_keywords"
    try:
      # Get Abstract
      abstract = parsed_bs.find(class_="card-body acl-abstract")
      abstract = re.search('<span>(.*)</span>', str(abstract)).group(1).strip()
    except:
      try:
        url = paper_url+".pdf"
        request = requests.get(url)
        filestream = io.BytesIO(request.content)
        pdf = fitz.open(stream=filestream, filetype="pdf")
        page = pdf.load_page(0)
        text = page.get_text("text")
        text_noline = re.sub("-\n"," ",text)
        text_noline = re.sub("\n"," ",text_noline)
        abstract = re.search('Abstract(.*)1 Introduction', text_noline).group(1).strip()
      except:
        abstract = "failed_to_scrap"

    # Append to the list
    list_year.append(year)
    list_title.append(title)
    list_conf_jour.append(conf_jour_name)
    list_authors.append(authors)
    list_keywords.append(keywords)
    list_abstract.append(abstract)
  
  except:
    list_year.append("no_metadata")
    list_title.append("no_metadata")
    list_conf_jour.append("no_metadata")
    list_authors.append("no_metadata")
    list_keywords.append("no_metadata")
    list_abstract.append("no_metadata")

  print(index)
  index += 1

In [5]:
# Concatenate list as new column
acl["Year"], acl["Title"], acl["Conf/Journal Name"] = list_year, list_title, list_conf_jour
acl["Authors"], acl["Keywords"], acl["Abstract"] = list_authors, list_keywords, list_abstract

In [7]:
acl.to_excel("./paper-metadata/PaperMetadata_Unclean_ACL.xlsx",index=False)